In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
!pip install tensorflow=='2.0.0'

In [ ]:
import tensorflow as tf
tf.version

In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
import cv2

In [ ]:
# Navigate to images directory
import os
images = os.listdir("../input/celeba-dataset/img_align_celeba/img_align_celeba")
trainData = []
print(type(images))

In [ ]:
len(images)

In [ ]:
for i in range(300):
    trainData.append(cv2.resize(
        cv2.imread("../input/celeba-dataset/img_align_celeba/img_align_celeba/" + images[i]), (64,64)))

In [ ]:
trainData = np.float32((np.array(trainData) - 127.5) / 127.5)

In [ ]:
len(trainData)

**Generator**

The generator uses upsampling (Conv2DTranspose) to produce an image from a seed(random noise). Start with a Dense layer that takes this seed as input, then upsample several times until you reach the desired image size of 64x64x3.

In [ ]:
def generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(4*4*1024, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Reshape((4, 4, 1024)))
    assert model.output_shape == (None, 4, 4, 1024)
    
    model.add(layers.Conv2DTranspose(512, (4,4), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    assert model.output_shape == (None, 8, 8, 512)
    
    model.add(layers.Conv2DTranspose(256, (4,4), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    assert model.output_shape == (None, 16, 16, 256)
    
    model.add(layers.Conv2DTranspose(128, (4,4), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    assert model.output_shape == (None, 32, 32, 128)
    
    model.add(layers.Conv2DTranspose(3,
                                     (4,4),
                                     strides=(2, 2),
                                     padding='same',
                                     use_bias=False,
                                     activation='tanh'))
    assert model.output_shape == (None, 64, 64, 3)
    return model

In [ ]:
generator = generator_model()
tf.keras.utils.plot_model(generator, show_shapes=True, to_file='generator.png')
# gen_model.summary()

In [ ]:
noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0], cmap='gray')

**Discriminator**

The discriminator is a CNN-based image classifier.

In [ ]:
def discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(128,
                            (4,4),
                            strides=(2,2),
                            padding='same',
                            input_shape=[64, 64, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(256, (4,4), strides=(2,2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv2D(512, (4,4), strides=(2,2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv2D(1024, (4,4), strides=(2,2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(1, (4,4), strides=(1,1), padding='same'))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [ ]:
discriminator = discriminator_model()
tf.keras.utils.plot_model(discriminator, to_file='discriminator.png', show_shapes=True)

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

**Discriminator Loss**

This method tells how well the discriminator is able to distinguish between real and generated images. It compares the discrimators's preds on real images to an array of 1's, and the predictions of fake(generated) images to an array of zeros.

In [ ]:
# discriminator loss
def discriminator_loss(real_out, fake_out):
    real_loss = cross_entropy(tf.ones_like(real_out), real_out)
    fake_loss = cross_entropy(tf.zeros_like(fake_out), fake_out)
    total_loss = real_loss + fake_loss
    return total_loss

**Generator Loss**

THe generator loss tells how well it was able to fool the discriminator. If the generator is performing well then the discriminator will classify generated images as real. Here we will compare the discriminators decisions on the generated images to the array of 1's. 

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
# define separate optimizers for both networks
generator_opt = tf.keras.optimizers.Adam(1e-4)
discriminator_opt = tf.keras.optimizers.Adam(1e-4)

**Save Checkpoints**

If the notebook/training is interrupted checkpoints can be restored.

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_opt=generator_opt,
                                 discriminator_opt=discriminator_opt,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16

# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([10, noise_dim])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        
        g_loss = generator_loss(fake_output)
        d_loss = discriminator_loss(real_output, fake_output)
    
    gen_grads = gen_tape.gradient(g_loss, generator.trainable_variables)
    dis_grads = disc_tape.gradient(d_loss, discriminator.trainable_variables)
    
    generator_opt.apply_gradients(zip(gen_grads, generator.trainable_variables))
    discriminator_opt.apply_gradients(zip(dis_grads, discriminator.trainable_variables))

In [ ]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        
        for i in range(30):
            train_step(dataset[i:i+10])
        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
    noise = tf.random.normal([10,100])
    generator(noise, training=False)
    generated = generator(noise, training=False)
    tensor = generated[0,:,:,0]
    tensorInArr = tf.keras.backend.get_value(tensor)
    plt.imshow(tensorInArr, cmap='gray')
#         print("epoch {}".format(epoch))
#         # Produce images for the GIF as we go
#         display.clear_output(wait=True)
#         generate_and_save_images(generator,
#                              epoch + 1,
#                              seed)

#         # Save the model every 15 epochs
#         if (epoch + 1) % 15 == 0:
#             checkpoint.save(file_prefix = checkpoint_prefix)

#         print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

#     # Generate after the final epoch
#     display.clear_output(wait=True)
#     generate_and_save_images(generator,
#                            epochs,
#                            seed)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4,4))   
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')
  
    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show

In [ ]:
train(trainData, 25)